In [1]:
import numpy as np
import random as rd
import scipy.sparse as sp
from time import time
import pickle
import os
import pandas as pd

In [2]:
train_file = 'train.txt'
test_file = 'test.txt'
n_users, n_items = 0, 0
n_train, n_test = 0, 0
neg_pools = {}

exist_users = []

with open(train_file) as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            items = [int(i) for i in l[1:]]
            uid = int(l[0])
            exist_users.append(uid)
            n_items = max(n_items, max(items))
            n_users = max(n_users, uid)
            n_train += len(items)

with open(test_file) as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n')
            try:
                items = [int(i) for i in l.split(' ')[1:]]
            except Exception:
                continue
            n_items = max(n_items, max(items))
            n_test += len(items)

In [3]:
n_items += 1
n_users += 1
R = sp.dok_matrix((n_users, n_items), dtype=np.float32)
train_items, test_set = {}, {}

In [4]:
with open(train_file) as f_train:
    for l in f_train.readlines():
        if len(l) == 0:
            break
        l = l.strip('\n')
        items = [int(i) for i in l.split(' ')]
        uid, user_items = items[0], items[1:]

        for i in train_items:
            R[uid, i] = 1

        train_items[uid] = user_items

IndexError: column index (3706) out of range

In [ ]:
uidW, iidW = None, None

In [ ]:
def E_score2(a, b):
    return np.sum(np.power(a-b, 2))

In [ ]:
def data_partition_1(train_items, k=10, T=50):
    uidW = pd.read_pickle("user_pretrain.pk")
    iidW = pd.read_pickle("item_pretrain.pk")

    # get_data_interactions_1
    data = []
    for i in train_items:
        for j in train_items[i]:
            data.append([i, j])

    # Randomly select k centroids
    max_data = 1.2 * len(data) / k
    centroids = rd.sample(data, k)

    # centro emb

    centroembs = []
    for i in range(k):
        temp_u = uidW[centroids[i][0]]
        temp_i = iidW[centroids[i][1]]
        centroembs.append([temp_u, temp_i])

    for _ in range(T):
        C = [{} for _ in range(k)]
        C_num = [0 for _ in range(k)]
        Scores = {}
        for i in range(len(data)):
            for j in range(k):

                score_u = E_score2(uidW[data[i][0]], centroembs[j][0])
                score_i = E_score2(iidW[data[i][1]], centroembs[j][1])

                Scores[i, j] = -score_u * score_i

        Scores = sorted(Scores.items(), key=lambda x: x[1], reverse=True)

        fl = set()
        for i in range(len(Scores)):
            if Scores[i][0][0] not in fl:

                if C_num[Scores[i][0][1]] < max_data:
                    if data[Scores[i][0][0]][0] not in C[Scores[i][0][1]]:
                        C[Scores[i][0][1]][data[Scores[i][0][0]][0]] = [
                            data[Scores[i][0][0]][1]]
                    else:
                        C[Scores[i][0][1]][data[Scores[i][0][0]]
                                           [0]].append(data[Scores[i][0][0]][1])
                    fl.add(Scores[i][0][0])
                    C_num[Scores[i][0][1]] += 1

        centroembs_next = []
        for i in range(k):
            temp_u = []
            temp_i = []

            for j in C[i].keys():
                for l in C[i][j]:
                    temp_u.append(uidW[j])
                    temp_i.append(iidW[l])
            centroembs_next.append([np.mean(temp_u), np.mean(temp_i)])

        loss = 0.0

        for i in range(k):
            score_u = E_score2(centroembs_next[i][0], centroembs[i][0])

            score_i = E_score2(centroembs_next[i][1], centroembs[i][1])

            loss += (score_u * score_i)

        centroembs = centroembs_next
        for i in range(k):
            print(C_num[i])

        print(_, loss)

    users = [[] for i in range(k)]
    items = [[] for i in range(k)]

    for i in range(k):
        users[i] = C[i].keys()
        for j in C[i].keys():
            for l in C[i][j]:
                if l not in items[i]:
                    items[i].append(l)

    return C, users, items

In [ ]:
C, users, items = data_partition_1(train_items)

IndexError: index 10215 is out of bounds for axis 0 with size 3707